In [11]:
import numpy as np
import timeit
from beiras_aux import LoadText,predict_next_chars,print_predicctions


In [4]:
window_size = 100
step_size = 1
X,y,chars,chars_to_indices,indices_to_chars,text_clean=LoadText('Beiras.txt',window_size,step_size);

In [5]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/cpu:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9083848884202467640, name: "/gpu:0"
 device_type: "GPU"
 memory_limit: 3868721152
 locality {
   bus_id: 1
 }
 incarnation: 1496205731994607219
 physical_device_desc: "device: 0, name: GRID K520, pci bus id: 0000:00:03.0"]

In [8]:
def add_log(log_string):
    with open("log-hyper.txt", "a") as myfile:
        myfile.write(log_string + "\n")  
        

In [9]:
### necessary functions from the keras library
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import keras
import random
from keras.layers import Dense, Activation,GRU
def create_gru_model(chars):
    num_chars = len(chars)
    model= Sequential()
    # 1 Layer .- LSTM layer 1 should be an LSTM module with 200 hidden units
    model.add(GRU(200,input_shape = (window_size,num_chars),return_sequences=True))
    # 2 Layer .-  Dense, with number chars unit and softmax activation
    model.add(GRU(200))
    model.add(Dense(num_chars,activation='softmax'))
    # initialize optimizer
    optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
    # compile model --> make sure initialized optimizer and callbacks - as defined above - are used
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)
    return model

Using TensorFlow backend.


loss='categorical_crossentropy'
RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

sobre 1225 s train

loss='categorical_crossentropy'
RMSprop(lr=0.01, rho=0.9, epsilon=1e-08, decay=0.0)

In [6]:
epoch=30

In [13]:
class LossHistory(keras.callbacks.Callback):
    def on_epoch_end(self, batch, logs={}):
        add_log(str(logs))
history = LossHistory()
import timeit

In [ ]:
add_log("\n\nlr 0.001 RMSprot bacch 500")
model=create_gru_model(chars)
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
# compile model --> make sure initialized optimizer and callbacks - as defined above - are used
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
start = timeit.default_timer()
model.fit(X, y, batch_size=500, nb_epoch=epoch,verbose = 1,callbacks=[history])
stop = timeit.default_timer()
add_log("time :"  + str(stop - start ))
model.save_weights('model_weights/best_beiras_gru_textdata_weights-lr001.hdf5')
print_predicctions(model,'model_weights/best_beiras_gru_textdata_weights-lr001.hdf5')


/home/aind2/anaconda3/envs/beiras-rnn/lib/python3.6/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/30
 547000/1174280 [============>.................] - ETA: 645s - loss: 2.0926 

In [ ]:
add_log("\n\nlr 0.01 RMSprot bacch 500")
model=create_gru_model(chars)
optimizer = keras.optimizers.RMSprop(lr=0.01, rho=0.9, epsilon=1e-08, decay=0.0)
# compile model --> make sure initialized optimizer and callbacks - as defined above - are used
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
start = timeit.default_timer()
model.fit(X, y, batch_size=500, nb_epoch=epoch,verbose = 1,callbacks=[history])
stop = timeit.default_timer()
add_log("time :"  + str(stop - start ))
model.save_weights('model_weights/best_beiras_gru_textdata_weights-lr01.hdf5')
print_predicctions(model,'model_weights/best_beiras_gru_textdata_weights-lr01.hdf5')



In [ ]:
add_log("\n\nlr 0.1 RMSprot bacch 500")
model=create_gru_model(chars)
optimizer = keras.optimizers.RMSprop(lr=0.1, rho=0.9, epsilon=1e-08, decay=0.0)
# compile model --> make sure initialized optimizer and callbacks - as defined above - are used
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
start = timeit.default_timer()
model.fit(X, y, batch_size=500, nb_epoch=epoch,verbose = 1,callbacks=[history])
stop = timeit.default_timer()
add_log("time :"  + str(stop - start ))
model.save_weights('model_weights/best_beiras_gru_textdata_weights-lr1.hdf5')
print_predicctions(model,'model_weights/best_beiras_gru_textdata_weights-lr1.hdf5')

In [ ]:
add_log("\n\nlr 0.001 Adam bacch 500")
model=create_gru_model(chars)
optimizer = keras.optimizers.Adam(lr=0.001)
# compile model --> make sure initialized optimizer and callbacks - as defined above - are used
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
start = timeit.default_timer()
model.fit(X, y, batch_size=500, nb_epoch=epoch,verbose = 1,callbacks=[history])
stop = timeit.default_timer()
add_log("time :"  + str(stop - start ))
model.save_weights('model_weights/best_beiras_gru_textdata_weights-adam-lr001.hdf5')
print_predicctions(model,'model_weights/best_beiras_gru_textdata_weights-adam-lr001.hdf5')

/home/aind2/anaconda3/envs/beiras-rnn/lib/python3.6/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/30
 584000/1174280 [=============>................] - ETA: 614s - loss: 2.0793 

In [ ]:
add_log("\n\nlr 0.001 RSProp bacch 32")
model=create_gru_model(chars)
optimizer = keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
# compile model --> make sure initialized optimizer and callbacks - as defined above - are used
model.compile(loss='categorical_crossentropy', optimizer=optimizer)
start = timeit.default_timer()
model.fit(X, y, batch_size=32, nb_epoch=epoch,verbose = 1,callbacks=[history])
stop = timeit.default_timer()
add_log("time :"  + str(stop - start ))
model.save_weights('model_weights/best_beiras_gru_textdata_weights-batch32.hdf5')
print_predicctions(model,'model_weights/best_beiras_gru_textdata_weights-batch32.hdf5')

/home/aind2/anaconda3/envs/beiras-rnn/lib/python3.6/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/30
  24256/1174280 [..............................] - ETA: 11939s - loss: 2.4379